* ~~get json data from endpoint /themes~~
* ~~create db table /themes~~
* ~~put data to df /themes~~
* add/update records in table /themes
* create db table /sets
* get latest timestamp from sets table /sets
* get json data from endpoint, order from newest to latest update /sets
* add to df only records latest then in table /sets 
* put data to df /sets
* get sets/{set_num}/parts for each set from df and put it to parts df /sets
* add/update records in table /sets
* create db table /parts
* add/update records in table /parts 

In [1]:
from csv import *
from db_handler import *
from api_handler import *
import json
import pandas as pd

In [2]:
keys = json.load(open('keys.json'))
API_KEY = keys['API_KEY']

get json data from themes endpoint

In [3]:
endpoint = 'themes'
dfs = {}
dfs[f'{endpoint}'] = get_df_from_api(endpoint,API_KEY,'50')

db connection string

In [4]:
connection_data = json.load(open('connection_data.json'))
host_name = connection_data['host_name']
dbname = connection_data['dbname']
username = connection_data['username']
password = connection_data['password']
port = connection_data['port']
conn = None

conn = connect_to_db(host_name, dbname, username, password, port)
curr = conn.cursor()

create themes table

In [ ]:
themes_table_statement = create_table_statement(dfs['themes'],'themes')
create_table(curr, themes_table_statement)
conn.commit()

insert themes data into database

In [5]:
insert_into_table_themes(curr, dfs['themes'])
conn.commit()

In [ ]:
def insert_into_table(curr,dataframe):
    insert_statement = ("""INSERT INTO themes (id, parent_id, name) VALUES(%s, %s, '%s');""")
    for i, row in dataframe.iterrows():
        id = None if str(row['id']) == 'nan' else row['id']
        parent_id = None if str(row['parent_id']) == 'nan' else row['parent_id']
        name = None if str(row['name']) == 'nan' else row['name']
        rows_to_insert = (id, parent_id, name)
        print(insert_statement % rows_to_insert)

insert_into_table(curr,dfs['themes'])